In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import random
from datetime import datetime
import time

import matplotlib.pyplot as plt

pi = 3.14159265359

maxval=1e9
minval=1e-9



2025-06-10 01:31:23.306084: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-10 01:31:23.306183: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-10 01:31:23.384683: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 01:31:23.548854: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-10 01:31:36.352149: W tensorflow/compiler/tf2

In [2]:
# os.chdir('SmartPix/data_generator')
os.chdir('SmartPix/BetterDG')
!pwd

/home/das214/SmartPix/BetterDG


In [3]:
!ls

codesign_catapult.ipynb       preselection_processor.py
docs			      __pycache__
evaluate.py		      requirements.txt
from_weights.ipynb	      site
loss.py			      test_run.py
Merge_plan_lab.ipynb	      trained_models
mergeplan.py		      train_model_Better_DG.ipynb
mkdocs.yml		      train_model_new_example.ipynb
model_batchnorm		      train_model_optimized_giuseppe.ipynb
models.py		      train_model_optimized.ipynb
OptimizedDataGeneratorNew.py  train_model.py
OptimizedDataGenerator.py     utils.py
OptimizedDataGenerator_v2.py  xysum_model.py
plotting


In [4]:
import importlib
import OptimizedDataGenerator_v2

importlib.reload(OptimizedDataGenerator_v2)
from OptimizedDataGenerator_v2 import OptimizedDataGenerator


In [5]:
#from dataprep import *
# from OptimizedDataGenerator_v2 import OptimizedDataGenerator
from loss import *
from models import *

In [6]:
dataset_base_dir = "/depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/"
tfrecords_base_dir = os.path.join(dataset_base_dir, "TFR_files", "2t")

dataset_base_dir = os.path.join(dataset_base_dir, "parquets")
tfrecords_dir_train = os.path.join(tfrecords_base_dir, "TFR_train")
tfrecords_dir_val   = os.path.join(tfrecords_base_dir, "TFR_val")

batch_size = 5000
val_batch_size = 5000
train_file_size = 75
val_file_size = 25

In [7]:
start_time = time.time()
validation_generator = OptimizedDataGenerator(
    dataset_base_dir = dataset_base_dir,
    file_type = "parquet",
    data_format = "3D",
    batch_size = val_batch_size,
    # optimize_batch_size = True,
    file_count = val_file_size,
    to_standardize= True,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    input_shape = (2,13,21), # (20,13,21),
    transpose = (0,2,3,1),
    shuffle = False, 
    files_from_end=True,

    tfrecords_dir = tfrecords_dir_val,
    use_time_stamps = [0,19],
    max_workers = 2
)

print("--- Validation generator %s seconds ---" % (time.time() - start_time))

# training generator
start_time = time.time()
training_generator = OptimizedDataGenerator(
    dataset_base_dir = dataset_base_dir,
    file_type = "parquet",
    data_format = "3D",
    batch_size = batch_size,
    # optimize_batch_size = True,
    file_count = train_file_size,
    to_standardize= True,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    input_shape = (2,13,21), # (20,13,21),
    transpose = (0,2,3,1),
    shuffle = False, # True 

    tfrecords_dir = tfrecords_dir_train,
    use_time_stamps = [0,19],
    max_workers = 2
)
print("--- Training generator %s seconds ---" % (time.time() - start_time))

Processing Files...: 100%|██████████| 25/25 [00:16<00:00,  1.56it/s]


Directory /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_val is removed...


Saving batches as TFRecords: 100%|██████████| 102/102 [00:24<00:00,  4.15it/s]


Metadata saved successfully ast /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_val/metadata.json
Loading metadata from /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_val/metadata.json
--- Validation generator 41.25817394256592 seconds ---


Processing Files...: 100%|██████████| 75/75 [00:45<00:00,  1.63it/s]


Directory /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_train is removed...


Saving batches as TFRecords: 100%|██████████| 306/306 [01:13<00:00,  4.17it/s]


Metadata saved successfully ast /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_train/metadata.json
Loading metadata from /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_train/metadata.json


--- Training generator 120.27378225326538 seconds ---


In [8]:
# Loading pre-generated TFRecords
validation_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir= tfrecords_dir_val,
    shuffle=True,
    seed=42,
    quantize=True,
)

training_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir = tfrecords_dir_train,
    shuffle=True,
    seed=42,
    quantize=True,
)


Loading metadata from /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_val/metadata.json


Loading metadata from /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_train/metadata.json


In [9]:
model=CreateModel((13,21,2),n_filters=5,pool_size=3)
model.compile(
    optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-3),
    loss=custom_loss
)

model.summary()

2025-06-10 01:35:00.876649: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 21, 2)]       0         
                                                                 
 q_separable_conv2d (QSepar  (None, 11, 19, 5)         33        
 ableConv2D)                                                     
                                                                 
 q_activation (QActivation)  (None, 11, 19, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 11, 19, 5)         30        
                                                                 
 q_activation_1 (QActivatio  (None, 11, 19, 5)         0         
 n)                                                              
                                                                 
 average_pooling2d (Average  (None, 3, 6, 5)           0     

In [10]:
from datetime import datetime

fingerprint = '%08x' % random.randrange(16**8)
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
os.makedirs("trained_models", exist_ok=True)
base_dir = f'./trained_models/model-{fingerprint}-checkpoints'
os.makedirs(base_dir, exist_ok=True)  
checkpoint_filepath = base_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'

In [11]:
print(fingerprint)

3f76ef3f


In [12]:
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, Callback

early_stopping_patience = 50

class CustomModelCheckpoint(ModelCheckpoint):
    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch, logs)
        checkpoints = [f for f in os.listdir(base_dir) if f.startswith('weights')]
        if len(checkpoints) > 1:
            checkpoints.sort()
            for checkpoint in checkpoints[:-1]:
                os.remove(os.path.join(base_dir, checkpoint))

es = EarlyStopping(patience=early_stopping_patience, restore_best_weights=True)

mcp = CustomModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=True,
    save_freq='epoch',
    verbose=1
)

csv_logger = CSVLogger(f'{base_dir}/training_log.csv', append=True)

In [ ]:
history = model.fit(
        x=training_generator,
        validation_data=validation_generator,
        callbacks=[es, mcp, csv_logger],
        epochs=1000,
        shuffle=False,
        verbose=1
    )

In [ ]:
'''
Epoch 1/1000
305/305 [==============================] - ETA: 0s - loss: 27173.7227
Epoch 1: val_loss improved from inf to 3957.69629, saving model to ./trained_models/model-e0ef8b33-checkpoints/weights.01-t27173.72-v3957.70.hdf5
305/305 [==============================] - 34s 102ms/step - loss: 27173.7227 - val_loss: 3957.6963
Epoch 2/1000
305/305 [==============================] - ETA: 0s - loss: 2077.7952
Epoch 2: val_loss improved from 3957.69629 to -473.36844, saving model to ./trained_models/model-e0ef8b33-checkpoints/weights.02-t2077.80-v-473.37.hdf5
305/305 [==============================] - 30s 97ms/step - loss: 2077.7952 - val_loss: -473.3684
Epoch 3/1000
305/305 [==============================] - ETA: 0s - loss: -935.6221
Epoch 3: val_loss improved from -473.36844 to -1284.97668, saving model to ./trained_models/model-e0ef8b33-checkpoints/weights.03-t-935.62-v-1284.98.hdf5
305/305 [==============================] - 61s 199ms/step - loss: -935.6221 - val_loss: -1284.9767
Epoch 4/1000
305/305 [==============================] - ETA: 0s - loss: -484.1406
Epoch 4: val_loss improved from -1284.97668 to -1554.20630, saving model to ./trained_models/model-e0ef8b33-checkpoints/weights.04-t-484.14-v-1554.21.hdf5
305/305 [==============================] - 36s 116ms/step - loss: -484.1406 - val_loss: -1554.2063
Epoch 5/1000
305/305 [==============================] - ETA: 0s - loss: -3135.4961
Epoch 5: val_loss improved from -1554.20630 to -2936.62402, saving model to ./trained_models/model-e0ef8b33-checkpoints/weights.05-t-3135.50-v-2936.62.hdf5
305/305 [==============================] - 40s 130ms/step - loss: -3135.4961 - val_loss: -2936.6240
Epoch 6/1000
 38/305 [==>...........................] - ETA: 19s - loss: -3885.3333
'''